In [ ]:
from futu import *
import time
import pandas as pd

# 创建连接
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

# 创建过滤器列表
filter_list = []

# 1. 市值 > 100亿
market_val_filter = SimpleFilter()
market_val_filter.filter_min = 100 * 100000000  # 100亿
market_val_filter.filter_max = float('inf')
market_val_filter.stock_field = StockField.MARKET_VAL
market_val_filter.is_no_filter = False
market_val_filter.sort = SortDir.DESCEND  # 按市值降序排列
filter_list.append(market_val_filter)

# 2. PE TTM在8-20之间
pe_ttm_filter = SimpleFilter()
pe_ttm_filter.filter_min = 8
pe_ttm_filter.filter_max = 20
pe_ttm_filter.stock_field = StockField.PE_TTM
pe_ttm_filter.is_no_filter = False
filter_list.append(pe_ttm_filter)

# 3. 资产负债率 < 60%
debt_rate_filter = SimpleFilter()
debt_rate_filter.filter_min = 0
debt_rate_filter.filter_max = 60
debt_rate_filter.stock_field = StockField.DEBT_ASSET_RATE
debt_rate_filter.is_no_filter = False
filter_list.append(debt_rate_filter)

# 4. 净利率 > 10%
net_profit_rate_filter = SimpleFilter()
net_profit_rate_filter.filter_min = 10
net_profit_rate_filter.filter_max = float('inf')
net_profit_rate_filter.stock_field = StockField.NET_PROFIT_RATE
net_profit_rate_filter.is_no_filter = False
filter_list.append(net_profit_rate_filter)

# 5. ROE > 8%
roe_filter = SimpleFilter()
roe_filter.filter_min = 8
roe_filter.filter_max = float('inf')
roe_filter.stock_field = StockField.RETURN_ON_EQUITY_RATE
roe_filter.is_no_filter = False
filter_list.append(roe_filter)

# 6. 流动比率 > 1.5
current_ratio_filter = SimpleFilter()
current_ratio_filter.filter_min = 1.5
current_ratio_filter.filter_max = float('inf')
current_ratio_filter.stock_field = StockField.CURRENT_RATIO
current_ratio_filter.is_no_filter = False
filter_list.append(current_ratio_filter)

# 7. 振幅 < 3%
amplitude_filter = SimpleFilter()
amplitude_filter.filter_min = 0
amplitude_filter.filter_max = 3
amplitude_filter.stock_field = StockField.AMPLITUDE
amplitude_filter.is_no_filter = False
filter_list.append(amplitude_filter)

# 执行选股
nBegin = 0
last_page = False
all_stocks = []

try:
    while not last_page:
        ret, ls = quote_ctx.get_stock_filter(market=Market.HK, filter_list=filter_list, begin=nBegin, num=200)
        if ret == RET_OK:
            last_page, all_count, ret_list = ls
            print(f'获取结果: {len(ret_list)}个股票, 总计: {all_count}个')
            
            # 收集符合条件的股票
            for item in ret_list:
                stock_info = {
                    '股票代码': item.stock_code,
                    '股票名称': item.stock_name,
                    '最新价': item.cur_price,
                    '市值(亿)': round(item[market_val_filter] / 100000000, 2),
                    'PE TTM': round(item[pe_ttm_filter], 2),
                    '资产负债率(%)': round(item[debt_rate_filter], 2),
                    '净利率(%)': round(item[net_profit_rate_filter], 2),
                    'ROE(%)': round(item[roe_filter], 2),
                    '流动比率': round(item[current_ratio_filter], 2),
                    '振幅(%)': round(item[amplitude_filter], 2)
                }
                all_stocks.append(stock_info)
            
            # 更新偏移量
            nBegin += len(ret_list)
            
            # 避免触发频率限制
            if not last_page:
                time.sleep(3)
        else:
            print(f'查询失败: {ls}')
            break
    
    # 将结果转换为DataFrame并显示
    if all_stocks:
        result_df = pd.DataFrame(all_stocks)
        print("\n选股结果:")
        display(result_df)
    else:
        print("没有找到符合条件的股票")
        
except Exception as e:
    print(f"发生错误: {str(e)}")
    
finally:
    quote_ctx.close()  # 关闭连接

2025-05-18 16:33:35,382 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=780, host=127.0.0.1, port=11111, user_id=232724269
查询失败: 港股BMP权限下，不支持条件选股接口
没有找到符合条件的股票
查询失败: 港股BMP权限下，不支持条件选股接口
没有找到符合条件的股票


In [ ]:
from futu import *


2025-05-18 16:35:29,886 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=781, host=127.0.0.1, port=11111, user_id=232724269


In [25]:

quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data = quote_ctx.get_plate_list(Market.HK, Plate.CONCEPT)
if ret == RET_OK:
    print(data)
    print(data['plate_name'][0])    # 取第一条的板块名称
    print(data['plate_name'].values.tolist())   # 转为 list
else:
    print('error:', data)
quote_ctx.close()  # 结束后记得关闭当条连接，防止连接条数用尽

2025-05-18 16:41:01,802 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=783, host=127.0.0.1, port=11111, user_id=232724269
             code   plate_name   plate_id
0     HK.LIST1110        阿里概念股   LIST1110
1     HK.LIST1161          双十一   LIST1161
2     HK.LIST1166         苹果概念   LIST1166
3     HK.LIST1175         一带一路   LIST1175
4     HK.LIST1176         5G概念   LIST1176
..            ...          ...        ...
101  HK.LIST22928     美国降息利好概念  LIST22928
102  HK.LIST23586  DeepSeek概念股  LIST23586
103  HK.LIST23589      智能驾驶概念股  LIST23589
104  HK.LIST23593       机器人概念股  LIST23593
105  HK.LIST23598      AI医疗概念股  LIST23598

[106 rows x 3 columns]
阿里概念股
['阿里概念股', '双十一', '苹果概念', '一带一路', '5G概念', '粤港澳大湾区', '特斯拉概念股', '啤酒', '体育用品', '稀土概念', '腾讯概念', '云办公', 'SaaS概念', '在线教育', '汽车经销商', '核电', '化妆品', '石油股', '电讯设备', '电力股', '手游股', '婴童用品股', '百货业股', '港口运输股', '电信股', '环保', '煤炭股', '综合车企股', '电池', '物流', '内地物业管理股', '农业股', '黄金股', '奢侈品品牌股', '电力设备股', '重型机械股', '食品股', '保险股', '纸业股

# 将全部data存到一个 csv 文件里
data.to_csv('hk_plate_list.csv', index=False, encoding='utf-8-sig')

In [ ]:
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

ret, dataLIST1228 = quote_ctx.get_plate_stock('HK.LIST1228')
if ret == RET_OK:
    print(dataLIST1228)
    print(dataLIST1228['stock_name'][0])    # 取第一条的股票名称
    print(dataLIST1228['stock_name'].values.tolist())   # 转为 list
    data.to_csv('​LIST1228.csv', index=False, encoding='utf-8-sig')
else:
    print('error:', dataLIST1228)
quote_ctx.close()  # 结束后记得关闭当条连接，防止连接条数用尽

2025-05-18 16:53:39,816 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=785, host=127.0.0.1, port=11111, user_id=232724269
        code  lot_size stock_name  stock_owner  stock_child_type stock_type  \
0   HK.00376      2000       云锋金融          NaN               NaN      STOCK   
1   HK.00945       100     宏利金融-S          NaN               NaN      STOCK   
2   HK.00966       200       中国太平          NaN               NaN      STOCK   
3   HK.01299       200       友邦保险          NaN               NaN      STOCK   
4   HK.01336       100       新华保险          NaN               NaN      STOCK   
5   HK.01339      1000   中国人民保险集团          NaN               NaN      STOCK   
6   HK.01508      1000      中国再保险          NaN               NaN      STOCK   
7   HK.02318       500       中国平安          NaN               NaN      STOCK   
8   HK.02328      2000       中国财险          NaN               NaN      STOCK   
9   HK.02378        50         保诚          NaN   

In [40]:

quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)  # 创建行情对象
ret, snapshotDataLIST1228 = (quote_ctx.get_market_snapshot(
    dataLIST1228['code'].values.tolist()))  # 获取港股 HK.00700 的快照数据
quote_ctx.close()  # 关闭对象，防止连接条数用尽

2025-05-18 17:10:45,551 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=787, host=127.0.0.1, port=11111, user_id=232724269


In [41]:
snapshotDataLIST1228.to_csv('LIST1228快照.csv', index=False, encoding='utf-8-sig')  # 保存快照数据到CSV文件

In [43]:
# get first 3 DESC by dividend_lfy_ratio 
snapshotDataLIST1228.sort_values(by='dividend_lfy_ratio', ascending=False, inplace=True)
top_3_dividend = snapshotDataLIST1228.head(3)
top_3_dividend

,code,name,update_time,last_price,open_price,high_price,low_price,prev_close_price,volume,turnover,...,after_change_rate,after_amplitude,overnight_price,overnight_high_price,overnight_low_price,overnight_volume,overnight_turnover,overnight_change_val,overnight_change_rate,overnight_amplitude
7,HK.02318,中国平安,2025-05-16 16:08:21,46.10,46.50,46.80,45.6,46.65,48140743,2.210848e+09,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
15,HK.82318,中国平安-R,2025-05-16 16:08:25,42.45,42.60,42.60,42.1,43.15,262000,1.107732e+07,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
13,HK.06963,阳光保险,2025-05-16 16:08:22,3.34,3.38,3.38,3.3,3.37,8807500,2.935483e+07,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [74]:
def get_top_dividend_stocks(plate_stock_code, top_n=3):
    """
    获取分红率最高的前N只股票
    :param snapshot_df: 股票快照数据
    :param top_n: 返回的股票数量
    :return: 分红率最高的前N只股票
    """
    quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

    ret, data = quote_ctx.get_plate_stock(plate_stock_code)
    if ret == RET_OK:
        print(data)
        stock_codes = data['code'].values.tolist()
        print(stock_codes)
        data.to_csv(plate_stock_code+'.csv', index=False, encoding='utf-8-sig')
    else:
        print('error:', data)
    # 批量获取快照数据
    snapshot_data_list = []
    batch_size = 20  # get_market_snapshot 支持最多200只股票
    for i in range(0, len(stock_codes), batch_size):
        print("from {}, total {}".format(i, len(stock_codes)))
        ret_code, ret_data = quote_ctx.get_market_snapshot(stock_codes[i:i + batch_size])
        if ret_code == 0:
            snapshot_data_list.append(ret_data)
        else:
            print(f"快照获取失败: {ret_data}")
        time.sleep(1)
    if snapshot_data_list:
        snapshot_data = pd.concat(snapshot_data_list, ignore_index=True)
        print(snapshot_data)
        snapshot_data.to_csv(plate_stock_code+'_snapshot.csv', index=False, encoding='utf-8-sig')
    else:
        print("未获取到快照数据")
        quote_ctx.close()
        return pd.DataFrame()
    quote_ctx.close()  # 结束后记得关闭当条连接，防止连接条数用尽

    snapshot_data.sort_values(by='dividend_lfy_ratio',
                              ascending=False, inplace=True)
    # 适合分析低风险稳定高股息股票的主要指标
    columns_of_interest = [
        'code', 'name', 'last_price', 'turnover_rate', 'suspension', 'listing_date',
        'total_market_val', 'net_asset', 'net_profit', 'earning_per_share',
        'outstanding_shares', 'circular_market_val', 'net_asset_per_share',
        'ey_ratio', 'pe_ratio', 'pb_ratio', 'pe_ttm_ratio',
        'dividend_ttm', 'dividend_ratio_ttm', 'dividend_lfy', 'dividend_lfy_ratio',
        'amplitude'
    ]
    # 只显示这些指标
    display_cols = [
        col for col in columns_of_interest if col in snapshot_data.columns]
    snapshot_data_dividend = snapshot_data[display_cols]
    snapshot_data_dividend.to_csv(plate_stock_code+'_snapshot_dividend.csv',
                         index=False, encoding='utf-8-sig')
    return snapshot_data_dividend.head(top_n)

In [ ]:

get_top_dividend_stocks('HK.LIST1207', top_n=3)

低风险、稳定且高股息的股票板块：

​​电力股​​ (LIST1207)
公用事业板块，需求稳定，现金流可靠，通常有较高股息。

​​保险股​​ (LIST1228)
金融行业中的保险公司，盈利模式稳定，分红比例较高。

​​内银股​​ (LIST1239)
内地大型银行股，业务成熟，利息收入稳定，股息率较高。

​​水务股​​ (LIST1231)
公用事业类，需求刚性，现金流和分红政策稳定。

​​公路及铁路股​​ (LIST1246)
基础设施行业，收入受经济周期影响小，分红能力强。

​​高股息概念​​ (LIST1344)
直接定位为高股息的板块，符合筛选条件。

​​香港本地银行股​​ (LIST1252)
本地银行业务稳定，股息派发较为持续。

​​燃气股​​ (LIST1245)
公用事业属性强，市场需求稳定，分红稳健。

In [79]:
get_top_dividend_stocks('HK.LIST1231', top_n=3)

2025-05-18 20:42:54,087 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=814, host=127.0.0.1, port=11111, user_id=232724269
       code  lot_size stock_name  stock_owner  stock_child_type stock_type  \
0  HK.00257      1000       光大环境          NaN               NaN      STOCK   
1  HK.00270      2000       粤海投资          NaN               NaN      STOCK   
2  HK.00371      2000     北控水务集团          NaN               NaN      STOCK   
3  HK.00855      2000       中国水务          NaN               NaN      STOCK   
4  HK.00895       200       东江环保          NaN               NaN      STOCK   
5  HK.01065      2000   天津创业环保股份          NaN               NaN      STOCK   
6  HK.01129      4000     中国水业集团          NaN               NaN      STOCK   
7  HK.01790      5000       达力环保          NaN               NaN      STOCK   
8  HK.06136      1000       康达环保          NaN               NaN      STOCK   

    list_time        stock_id  main_contract last_trade_ti

,code,name,last_price,turnover_rate,suspension,listing_date,total_market_val,net_asset,net_profit,earning_per_share,...,net_asset_per_share,ey_ratio,pe_ratio,pb_ratio,pe_ttm_ratio,dividend_ttm,dividend_ratio_ttm,dividend_lfy,dividend_lfy_ratio,amplitude
2,HK.00371,北控水务集团,2.44,0.248,False,1993-04-19,2.451373e+10,2.988866e+10,1.808390e+09,0.180,...,2.975,0.060,13.555,0.820,13.555,0.157,6.43,0.157,6.434,3.320
0,HK.00257,光大环境,3.65,0.094,False,1970-01-01,2.242186e+10,4.821007e+10,3.378636e+09,0.550,...,7.848,0.070,6.636,0.465,6.636,0.220,6.03,0.220,6.027,1.370
5,HK.01065,天津创业环保股份,3.42,0.229,False,1994-05-17,5.370830e+09,1.075422e+10,8.715820e+08,0.555,...,6.848,0.081,6.162,0.499,6.118,0.182,5.32,0.182,5.321,1.176


中国神华（-2%增长）: 9.786604831516254
中国海油（15%增长）: 21.11586696757482
